<a href="https://colab.research.google.com/github/aimakerco/ai-sneaker-designer-lucid/blob/master/AI_Maker_Sneaker_Designer_Feature_Inversion_with_Lucid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#AI Maker - Sneaker Designer - Feature Inversion with Lucid

![Original Converse Tennis Shoes - Source Wikipedia](https://aimaker.co/wp-content/uploads/AI-Maker-Converse-Original-220.png)

Original Tennis Shoe - Source Wikipedia

![AI-Maker-Converse Tennis Shoe Inspiration](https://aimaker.co/wp-content/uploads/AI-Maker-Converse-Sneakers-AI-220.png)

AI Maker - Converse Sneaker Inspiration

# Overview of AI Sneaker Maker

**This Notebook is based on the excellent notebook - [Lucid Notebook](https://colab.research.google.com/github/tensorflow/lucid/blob/master/notebooks/misc/feature_inversion_caricatures.ipynb)**

I made changes to it to make it easier to design things with.  You can upload fashion design images and run them through this notebook to come up with new design inspirations.

For more details on this Notebook check out our post - [AI Maker - How to Become a Fashion Designer In Under 15 Minutes using Artificial Intelligence](https://aimaker.co/how-to-become-fashion-designer-under-15-minutes-using-artificial-intelligence)

# Download Your Ideas
The images generated from this notebook are saved to the local system.  You can download them by selecting the Files tab.  The images are stored in the /content/AI-Maker-lucid-fashion/ directory.  
 
1.   Select the image then right click on it.
2.   Select Download to download the image to your local system.




# Original Notebook Directions
      "This notebook uses  [**Lucid**](https://github.com/tensorflow/lucid) to produce feature inversion caricatures that are similar in spirit to the activation grids in [The Building Blocks of Interpretability](https://distill.pub/2018/building-blocks/). 

      This is slightly similar to the technique described by [Mahendran and Vedaldi](https://arxiv.org/pdf/1412.0035.pdf). However, we use a dot product objective, instead of L2 difference, and use transformation robustness to reduce artifacts. 

      This notebook doesn't introduce the abstractions behind lucid; you may wish to also read the [Lucid tutorial](https://colab.research.google.com/github/tensorflow/lucid/blob/master/notebooks/tutorial.ipynb).

      **Note**: The easiest way to use this tutorial is as a colab notebook, which allows you to dive in with no setup. We recommend you enable a free GPU by going:

      > **Runtime**   →   **Change runtime type**   →   **Hardware Accelerator: GPU**"

## Install, Import, Load Model

In [2]:
# Install Lucid
!pip install --quiet lucid==0.2.3

# Import libraries
import numpy as np
import tensorflow as tf
import scipy.ndimage as nd

from google.colab import files

import lucid.modelzoo.vision_models as models
import lucid.optvis.objectives as objectives
import lucid.optvis.param as param
import lucid.optvis.render as render
import lucid.optvis.transform as transform
from lucid.misc.io import show, load, save
from lucid.misc.io.reading import read
import time

     |████████████████████████████████| 40kB 5.0MB/s 


In [0]:
# Import the InceptionV1 (GoogLeNet) model from the Lucid modelzoo

model = models.InceptionV1()
model.load_graphdef()

## Setup

In [0]:
def imgToModelSize(arr):
  # this line produces 224 size image (small)
  #W = model.image_shape[0]
  # Increased from 224 to 512 for larger image files
  W = 512
  w, h, _ = arr.shape
  s = float(W) / min(w,h)
  arr = nd.zoom(arr, [s, s, 1], mode="nearest")
  w, h, _ = arr.shape
  dw, dh = (w-W)//2, (h-W)//3
  return arr[dw:dw+W, dh:dh+W]

In [0]:
@objectives.wrap_objective
def dot_compare(layer, batch=1, cossim_pow=0):
  def inner(T):
    dot = tf.reduce_sum(T(layer)[batch] * T(layer)[0])
    mag = tf.sqrt(tf.reduce_sum(T(layer)[0]**2))
    cossim = dot/(1e-6 + mag)
    return dot * cossim ** cossim_pow
  return inner

In [0]:
def feature_inversion(img=None, layer=None, n_steps=512, cossim_pow=0.0):
  file_name = ''
  x = 0
  with tf.Graph().as_default(), tf.Session() as sess:
    img = imgToModelSize(img)
    x = x + 1
    
    objective = objectives.Objective.sum([
        1.0 * dot_compare(layer, cossim_pow=cossim_pow),
        objectives.blur_input_each_step(),
    ])

    t_input = tf.placeholder(tf.float32, img.shape)
    param_f = param.image(img.shape[0], decorrelate=True, fft=True, alpha=False)
    param_f = tf.stack([param_f[0], t_input])

    transforms = [
      transform.pad(8, mode='constant', constant_value=.5),
      transform.jitter(8),
      transform.random_scale([0.9, 0.95, 1.05, 1.1] + [1]*4),
      transform.random_rotate(range(-5, 5) + [0]*5),
      transform.jitter(2),
    ]

    T = render.make_vis_T(model, objective, param_f, transforms=transforms)
    loss, vis_op, t_image = T("loss"), T("vis_op"), T("input")

    tf.global_variables_initializer().run()
    for i in range(n_steps): _ = sess.run([vis_op], {t_input: img})

    result = t_image.eval(feed_dict={t_input: img})
    # set the file name with the path
    timestr = time.strftime("%Y%m%d-%H%M%S")
    
    file_name = '/content/AI-Maker-lucid-sneaker/lucid_sneaker_method_' + layer + '_image_' + '_cos_' + str(cossim_pow) + '_' + timestr + '.jpg'
    # save the graphic as a JPEG file
    save(result[0], file_name, quality=90)
    show(result[0])

# Designing with Neural Networks

In [0]:
# set the directory for the output images (inspirations)
!mkdir /content/AI-Maker-lucid-sneaker

layers = ['conv2d%d' % i for i in range(0, 3)] + ['mixed3a', 'mixed3b', 
                                                  'mixed4a', 'mixed4b', 'mixed4c', 'mixed4d', 'mixed4e',
                                                  'mixed5a', 'mixed5b']

#img = ''

In [0]:
# Get the URLs for 
urls = ["https://upload.wikimedia.org/wikipedia/commons/thumb/0/0c/Burnetie_and_socks.jpg/512px-Burnetie_and_socks.jpg",
        "https://upload.wikimedia.org/wikipedia/commons/0/0d/Bape_Roadsta_%282009-03-21_00.20.04_by_Christian_H.%29.jpg"]

for url in urls:
    try:
        print('Loading URL - ' + url)
        img = load(url)
        show(imgToModelSize(img))
        for layer in layers:
          print('Layer - ' + layer)
          feature_inversion(img, layer=layer)
          print ""
    except ValueError:
        print('********   Error Occurred on URL - ' + url + ' *************')
        pass


## Varying Cossine  on the Designs

The Cossine value forces the design to be closer to the orginal image.  This can help with new inspirations for design ideas.

In [0]:
print('Original Image')
show(imgToModelSize(img))
for cossim in [0.0, 0.5, 1.0, 2.0]:
  print cossim
  feature_inversion(img, layer='mixed4d', cossim_pow=cossim)
  print ""

# Upload your own Files for new Design Ideas

Process the Images through the Layers & Cossines to get the most Inspiration for new ideas.  You will need to limit the uploaded images to 640x640.  Larger images can cause the system to produce errors.  If you get an error try loading a lower resolution image.

In [0]:
# Upload your Own File(s)
uploaded = files.upload()
for fn in uploaded.keys():
  img = load(fn)
  show(imgToModelSize(img))
  for layer in layers:
    print layer
    feature_inversion(img, layer=layer)
    print ""
    for cossim in [0.0, 0.5, 1.0, 2.0]:
      print('Layer - ' + layer + ', Cossim - ' + str(cossim))
      feature_inversion(img, layer=layer, cossim_pow=cossim)
      print ""

In [0]:
# Extra Credit - here are some more dresses to design

urls = ["https://upload.wikimedia.org/wikipedia/commons/thumb/a/a5/Black_Converse_sneakers.JPG/512px-Black_Converse_sneakers.JPG",
        "https://upload.wikimedia.org/wikipedia/commons/4/49/OBJSark-1_White_and_Black.png",
        "https://upload.wikimedia.org/wikipedia/commons/3/36/AdidasSuperstarII.jpg"]


for url in urls:
    try:
        print('Loading URL - ' + url)
        img = load(url)
        show(imgToModelSize(img))
        for layer in layers:
          print layer
          feature_inversion(img, layer=layer)
          print ""
          for cossim in [0.0, 0.5, 1.0, 2.0]:
            print('Layer - ' + layer + ', Cossim - ' + str(cossim))
            feature_inversion(img, layer=layer, cossim_pow=cossim)
            print ""
    except ValueError:
        print('********   Error Occurred on URL - ' + url + ' *************')
        pass


# More Sneaker Images
Here are some additional sneaker images to play with. Add these to the **urls** list above to process them.  Make sure and enclose them in brackets [ ] and seperate each item with a comma.

"https://upload.wikimedia.org/wikipedia/commons/4/46/Lyfn011.png"

"https://upload.wikimedia.org/wikipedia/commons/thumb/c/c1/Karhu_shoe.jpg/512px-Karhu_shoe.jpg"

"https://upload.wikimedia.org/wikipedia/commons/thumb/1/14/Vans_womans_shoe.JPG/512px-Vans_womans_shoe.JPG"

# Example

urls = ["https://upload.wikimedia.org/wikipedia/commons/4/46/Lyfn011.png", "https://upload.wikimedia.org/wikipedia/commons/thumb/c/c1/Karhu_shoe.jpg/512px-Karhu_shoe.jpg"]
